In [1]:
import warnings
warnings.simplefilter("ignore")
import scipy #BSD 3-clause 
import numpy as np #BSD 3-clause 
import pandas as pd #BSD 3-clause 
from sklearn import feature_extraction,cross_validation,svm,linear_model,ensemble#BSD 3-clause
import xgboost# Apache-2 
import lightgbm#MIT
from bayes_opt import BayesianOptimization#MIT


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df=pd.read_csv("evo_train.csv")
test=pd.read_csv("evo_test.csv")
cat=pd.read_csv("categories.csv")

In [3]:
import re
test.head(20)

,NAME,id
0,"Пиво ""Рижское"" светлое 0,5л",0
1,плавленый сыр король сыров 80г,1
2,заказ имл 7500342101814,2
3,Мерриес трусики М58,3
4,лупа канц мал,4
5,Шампунь ЧЛ Укрепление и уход. 400мл,5
6,Серьги-пуссеты,6
7,Краска Igora Royal 8-4,7
8,Зубочистики косточка кальциевая вкус курицы д/...,8
9,Вешалка Кружева-5,9


In [16]:
cat.head(40)

,GROUP_ID,NAME,PATH
0,6,Продукты питания,Прод->Продукты питания
1,7,Табачные изделия\r,Непрод->Табачные изделия\r
2,9,н/д,н/д->н/д
3,12,"Бытовая техника, оборудование и электроника","Непрод->Бытовая техника, оборудование и электр..."
4,14,"Косметика, гигиена и парфюмерия","Непрод->Косметика, гигиена и парфюмерия"
5,15,Муз.инструменты,Непрод->Муз.инструменты
6,16,Канцтовары,Непрод->Канцтовары
7,17,Текстильные товары,Непрод->Текстильные товары
8,18,Спорттовары,Непрод->Спорттовары
9,19,Упаковка,Непрод->Упаковка


In [14]:
df.GROUP_ID.value_counts()#распределние классов

7     6045
6     6014
24    5975
14    5972
34    5963
35    5957
30    3050
26    2999
25    2961
29     639
28     633
9      626
37     618
17     616
18     613
16     612
20     608
59     607
27     606
19     604
22     603
21     587
52     586
60     584
36     572
12     568
38     394
15     235
32      13
Name: GROUP_ID, dtype: int64

In [4]:
X=df.NAME[df.GROUP_ID!=32]#выкинул самый маленький класс
X1=df.NAME
y=df.GROUP_ID[df.GROUP_ID!=32]#дало прирост в -0.1%
y1=df.GROUP_ID

In [77]:
re.findall('[а-я&А-Я]\w+',test.NAME[0])#разбирался с регулярками, дабы тюнить tfidf или word2vec, не понадобилось

['Пиво', 'Рижское', 'светлое']

In [75]:
%%time
q=[]
for i in test.NAME:
    q.append(re.findall(r'[а-я&А-Я]\w+',i))

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 145 ms


In [78]:
np.zeros(300)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [5]:
X.shape

(55847,)

In [7]:
feature_extraction.text.CountVectorizer()
tf=feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
X_tf=tf.fit_transform(X)
tes=tf.transform(test.NAME)
X_tf1=tf.fit_transform(X1)
tes1=tf.transform(test.NAME)


In [8]:
sm=svm.LinearSVC(loss="hinge")

In [98]:
%%time
sm=svm.LinearSVC(loss="hinge")
tf=feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
X_tf=tf.fit_transform(X)
tes=tf.transform(test.NAME)
print(cross_validation.cross_val_score(sm,X_tf,y,cv=5).mean())

0.933854139256
CPU times: user 1min 11s, sys: 156 ms, total: 1min 11s
Wall time: 1min 11s


In [99]:
%%time
sm=svm.LinearSVC()
tf=feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))#Путем проб, ошибок и перебора, остановился на
X_tf=tf.fit_transform(X)                                     #линейном svm под tfidf-ом
tes=tf.transform(test.NAME)
print(cross_validation.cross_val_score(sm,X_tf,y,cv=10).mean())

0.937113332872
CPU times: user 57.6 s, sys: 384 ms, total: 57.9 s
Wall time: 58 s


In [100]:
sm.fit(X_tf1,y1)#первые сносные ответы

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [101]:
y_pred=sm.predict(tes1)

df_sub = pd.DataFrame({ 'GROUP_ID':y_pred })

df_sub.to_csv('./sub_new1.csv')

In [15]:
df_sub.head()

,id,GROUP_ID
0,0,35
1,1,6
2,2,9
3,3,26
4,4,24


In [109]:
sm.fit(X_tf1,y1)
y_pred=sm.predict(tes1)
df_sub = pd.DataFrame({ 'id': test.id })
df_sub.loc[:,'GROUP_ID']=y_pred
df_sub.to_csv('./sub.csv', index=False,decimal=",")

In [47]:
from gensim.models import KeyedVectors

# думал использовать word2vec, но в корпусе что я нашел небыло собственных имен(Рижское и тд.)
# а учить свой, я пока не умею
modelvshe = KeyedVectors.load_word2vec_format('/home/stas/Notebooks/evotor/rus.bin', binary=True)

In [84]:
len(modelvshe.word_vec("пиво"))

300

In [86]:
%%time
g=[]
for i in q:
    z=np.zeros(300)
    for j in i:
        z+=modelvshe.word_vec(j.lower())
    g.append(z)
        

KeyError: "word 'рижское' not in vocabulary"

In [5]:
def svemer(tol,C,inte):
    ester=svm.LinearSVC(intercept_scaling=inte,tol=tol,C=C)#перевел кроссвал в функцию, дабы оптимизировать баесом
    
    tf=feature_extraction.text.TfidfVectorizer(sublinear_tf=True)
    X_es=tf.fit_transform(X1)
    
    return cross_validation.cross_val_score(ester,X_es,y1,cv=5,n_jobs=5).mean()

In [6]:
gp_params = {"alpha": 1e-5}

In [7]:
svcBO = BayesianOptimization(svemer,{'C': (0.3, 1.8), "tol":(0.0000001,0.1),"inte":(0.01,100)})
svcBO.explore({'C': [0.95,1.05], "tol":[0.00005,0.0005],'inte':[0.1,10]})

In [8]:
%%time

svcBO.maximize(n_iter=1000)#спустя 2 дня подсчетов улучшил показатели больще чем на процент

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |         C |      inte |       tol | 
    1 | 00m07s |    0.93784 |    0.9500 |    0.1000 |    0.0001 | 
    2 | 03m54s |    0.93797 |    1.0500 |   10.0000 |    0.0005 | 
    3 | 09m22s |    0.90659 |    1.2484 |   67.5554 |    0.0396 | 
    4 | 09m20s |    0.86978 |    0.9819 |   70.2946 |    0.0535 | 
    5 | 09m22s |    0.86526 |    0.3309 |   98.7534 |    0.0126 | 
    6 | 09m22s |    0.86483 |    0.3129 |   78.0204 |    0.0584 | 
    7 | 05m06s |    0.93811 |    1.6747 |   11.5125 |    0.0107 | 
Bayesian Optimization
-----------------------------------------------------------------
 Step |   Time |      Value |         C |      inte |       tol | 
    8 | 09m18s |    0.92968 |    0.3000 |   40.8711 |    0.0000 | 
    9 | 09m27s |    0.89012 |    1.8000 |   53.6870 |    0.1000 | 
   10 | 08m13s |    0.93163 |    0.3000 |   26.1430 |    0.0000 | 
   11 | 08m47s |    0.93473

  116 | 09m14s |    0.93818 |    1.3835 |   18.7123 |    0.0111 | 


KeyboardInterrupt: 

In [6]:
%%time
q=[]
for i in range(1,5):    
    ester=svm.LinearSVC(tol=0.0071,C=1.7990,intercept_scaling=19.9007) #подбиралл колличество n-gramm, т.к.
    tf=feature_extraction.text.TfidfVectorizer(sublinear_tf=True,ngram_range=(1,i))#баес их совсем плохо учитывал
    X_es=tf.fit_transform(X1)
    z=cross_validation.cross_val_score(ester,X_es,y1,cv=5,n_jobs=5).mean()
    q.append((z,i))
    print(z)

0.93793419447
0.935767549483
0.933207906154


KeyboardInterrupt: 

In [5]:
%%time
ester=svm.LinearSVC(tol=0.0976,C=1.7990,intercept_scaling=0.0283)#два наиболее различных, 
                                                                 #лучше всего себя показавших решения
    
tf = feature_extraction.text.TfidfVectorizer(sublinear_tf=True,ngram_range=(1,1))
X_es = tf.fit_transform(X1)
tes1=tf.transform(test.NAME)

ester.fit(X_es,y1)



CPU times: user 3.65 s, sys: 4 ms, total: 3.65 s
Wall time: 3.71 s


In [6]:
y_pred=ester.predict(tes1)

df_sub = pd.DataFrame({ 'GROUP_ID':y_pred })

df_sub.to_csv('./sub_newbayes.csv')

In [17]:
%%time
ester=svm.LinearSVC(tol=0.0975,C=1.349,intercept_scaling=15.6865)#даже интересно, какое в итоге победило
    
tf = feature_extraction.text.TfidfVectorizer(sublinear_tf=True,ngram_range=(1,1))
X_es = tf.fit_transform(X1)
tes1=tf.transform(test.NAME)

ester.fit(X_es,y1)

y_pred=ester.predict(tes1)

df_sub = pd.DataFrame({ 'GROUP_ID':y_pred })

df_sub.to_csv('./sub.csv')

CPU times: user 2min 57s, sys: 32 ms, total: 2min 57s
Wall time: 2min 57s
